In [1]:
import pandas as pd
from itertools import combinations

In [22]:
nvalues = pd.read_csv('../data/nutritional_values.csv')
recipes = pd.read_csv('../data/filtered_recipes_100.csv',sep='|')

In [23]:
df = pd.merge(recipes, nvalues, on='Recipe_id', how='right')

In [29]:
filtros = ['Cena','Entrante','Plato principal','Acompañamiento']
df = df[df['Tipo'].isin(filtros)]

In [30]:
wanted_cols =['Recipe_id','Categoria','energia','grasa','proteina','carbohidratos']
df = df[wanted_cols]
cc = list(combinations(df.Recipe_id,2)) #antes de esto hacer filtradosde comida y cena etc
df.set_index('Recipe_id',inplace=True)
df

,Categoria,energia,grasa,proteina,carbohidratos
Recipe_id,,,,,
74154,Recetas de Aperitivos y tapas,2761.935834,157.494375,194.583745,139.315964
74146,Recetas de Aperitivos y tapas,7069.600000,640.376000,144.868000,194.390900
74056,Recetas de Aperitivos y tapas,2355.060000,214.041700,19.874375,91.354000
74043,Recetas de Aperitivos y tapas,55.960000,2.923750,5.533000,1.835510
74020,Recetas de Aperitivos y tapas,714.260000,20.732000,17.247000,112.550000
...,...,...,...,...,...
9584,Recetas de Verduras,115.271250,8.252150,1.469375,8.835000
9110,Recetas de Verduras,899.006316,50.345149,47.644113,62.896002
9114,Recetas de Verduras,239.403614,10.907224,8.645025,23.228600


In [31]:
cc_cols = ['Recipe1','Recipe2'] #poner comida/cena whatever

combined_df = pd.DataFrame(columns = cc_cols)
combined_df['Recipe1'] = [c[0] for c in cc]
combined_df['Recipe2'] = [c[1] for c in cc]
combined_df

,Recipe1,Recipe2
0,74154,74146
1,74154,74056
2,74154,74043
3,74154,74020
4,74154,73919
...,...,...
4441685,9110,6618
4441686,9110,31007
4441687,9114,6618
4441688,9114,31007


In [33]:
macros = ['energia','grasa','proteina','carbohidratos']
for macro in macros:
    combined_df[macro] = [df[macro][c[1]]+df[macro][c[0]] for c in cc]
combined_df

,Recipe1,Recipe2,energia,grasa,proteina,carbohidratos
0,74154,74146,9831.535834,797.870375,339.451745,333.706864
1,74154,74056,5116.995834,371.536075,214.458120,230.669964
2,74154,74043,2817.895834,160.418125,200.116745,141.151474
3,74154,74020,3476.195834,178.226375,211.830745,251.865964
4,74154,73919,3812.655834,258.666175,206.817995,168.871964
...,...,...,...,...,...,...
4441685,9110,6618,1344.883816,75.276899,52.117863,113.546002
4441686,9110,31007,2425.652279,188.093066,93.760783,90.789029
4441687,9114,6618,685.281114,35.838974,13.118775,73.878600
4441688,9114,31007,1766.049577,148.655141,54.761695,51.121627


In [40]:
combined_df = combined_df[combined_df['energia']<2500]

In [41]:
combined_df.to_csv('../data/combined.csv', index=False)